<a href="https://colab.research.google.com/github/raquel1313/EvaluacionFML/blob/main/lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo lineal simple
Objetivo:

Predecir cuántos headshots por ronda (RoundHeadShots) realizará un jugador en Counter-Strike, utilizando un modelo de regresión basado en el modelo lineal simple.


In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración para visualizaciones
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import mean_absolute_error

In [ ]:
from google.colab import files
import pandas as pd # Importing the pandas library

uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";") # Now pd is defined and can be used

Saving Anexo ET_demo_round_traces_2022 (1).csv to Anexo ET_demo_round_traces_2022 (1).csv


In [ ]:
df.head()

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,TimeAlive,TravelledDistance,RLethalGrenadesThrown,RNonLethalGrenadesThrown,PrimaryAssaultRifle,PrimarySniperRifle,PrimaryHeavy,PrimarySMG,PrimaryPistol,FirstKillTime,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots,ArmaPrincipal,CounterTerrorist,Terrorist
0,600,de_inferno,Terrorist,1,4,1,False,True,False,False,51.120.248.995.704.500,10.083.140.737.457.000,0,4,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,750,4400,0,0,0,0,PrimaryPistol,0,1
1,601,de_inferno,Terrorist,1,4,1,False,True,False,False,4.348.662.552.266.170,60.960.883.184.688.400,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,800,4400,0,0,0,0,PrimaryPistol,0,1
2,602,de_inferno,Terrorist,1,4,1,False,True,False,False,3.735.469.847.054.540,5.642.252.383.582.750,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,1000,4400,0,0,0,0,PrimaryPistol,0,1
3,603,de_inferno,Terrorist,1,4,1,False,True,False,False,4.767.886.136.441.470,8.963.473.313.229.970,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,850,4400,0,0,0,0,PrimaryPistol,0,1
4,604,de_inferno,Terrorist,1,4,1,False,True,True,False,5.312.251.089.027.310,10.865.649.734.393.100,1,1,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,1000,4400,0,0,0,0,PrimaryPistol,0,1


#LA MISMA PREPARACION QUE EL ENTREGABLE 1#

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79157 entries, 0 to 79156
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   79157 non-null  int64  
 1   Map                          79157 non-null  object 
 2   Team                         79155 non-null  object 
 3   InternalTeamId               79157 non-null  int64  
 4   MatchId                      79157 non-null  int64  
 5   RoundId                      79157 non-null  int64  
 6   RoundWinner                  79157 non-null  object 
 7   MatchWinner                  79156 non-null  object 
 8   Survived                     79157 non-null  bool   
 9   AbnormalMatch                79157 non-null  bool   
 10  TimeAlive                    79157 non-null  object 
 11  TravelledDistance            79157 non-null  object 
 12  RLethalGrenadesThrown        79157 non-null  int64  
 13  RNonLethalGrenad

In [ ]:
# Crear columna con el arma más usada
df['ArmaPrincipal'] = df[['PrimaryAssaultRifle', 'PrimarySniperRifle', 'PrimarySMG', 'PrimaryHeavy', 'PrimaryPistol']].idxmax(axis=1)

# Ahora sí agrupamos las kills por ArmaPrincipal
kills_by_weapon = df.groupby('ArmaPrincipal')['MatchKills'].sum().reset_index()



In [ ]:
# Función para identificar outliers con el método IQR
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Detectar outliers en MatchKills
matchkills_outliers = detect_outliers(df, 'MatchKills')
print(f"Número de outliers en MatchKills: {len(matchkills_outliers)}")

# Detectar outliers en MatchHeadshots
matchheadshots_outliers = detect_outliers(df, 'MatchHeadshots')
print(f"Número de outliers en MatchHeadshots: {len(matchheadshots_outliers)}")

Número de outliers en MatchKills: 162
Número de outliers en MatchHeadshots: 724


In [ ]:
# Eliminación de outliers
# Guardar outliers en un nuevo DataFrame
outliers_combined = pd.concat([matchkills_outliers, matchheadshots_outliers]).drop_duplicates()

# Eliminar outliers del DataFrame original
df_cleaned = df[~df.index.isin(outliers_combined.index)]

# Imprimir resultados
print(f"Número total de outliers combinados: {len(outliers_combined)}")
print("\nDataFrame original (sin outliers):")


Número total de outliers combinados: 827

DataFrame original (sin outliers):


In [ ]:
#Para tratar la variable team de mejor manera, creamos una columna para cada tipo de Team
df['CounterTerrorist'] = (df['Team'] == 'CounterTerrorist').astype(int)
df['Terrorist'] = (df['Team'] == 'Terrorist').astype(int)



In [ ]:
#Creamos una copia
df2= df.copy()

In [ ]:
#Traemos las armas
df2.ArmaPrincipal.unique()

array(['PrimaryPistol', 'PrimaryAssaultRifle', 'PrimarySMG',
       'PrimarySniperRifle', 'PrimaryHeavy'], dtype=object)

In [ ]:
#Remplazamos los datos de modelo String por numericos
df2.ArmaPrincipal.replace({"PrimaryPistol":1, "PrimaryAssaultRifle":2, "PrimarySMG":3, "PrimarySniperRifle":4, "PrimaryHeavy":5}, inplace=True)


In [ ]:
#Traemos los mapas
df2.Map.unique()

array(['de_inferno', 'de_nuke', 'de_mirage', 'de_dust2'], dtype=object)

In [ ]:
#Remplazamos por datos numericos
df2.Map.replace({"de_inferno":1, "de_nuke":2, "de_mirage":3, "de_dust2":4}, inplace=True)


In [ ]:
#Borramos columna Team
df2 = df2.drop('Team', axis=1)


In [ ]:
# Primero, convertir la columna a string
df2['MatchWinner'] = df2['MatchWinner'].astype(str)

In [ ]:
# Reemplazar 'True' por 1, 'False' por 0, y 'nan' por 0
df2['MatchWinner'] = df2['MatchWinner'].replace({'True': 1, 'False': 0, 'nan': 0})

In [ ]:
# Finalmente convertir a int
df2['MatchWinner'] = df2['MatchWinner'].astype(int)

In [ ]:
#Traemos RounWinner
print(df2['RoundWinner'].unique())

['False' 'True' 'False4' True False]


In [ ]:
# Primero asegurarte de que todo sea string
df2['RoundWinner'] = df2['RoundWinner'].astype(str)

# Luego asignar 1 si contiene 'True', 0 si contiene 'False'
df2['RoundWinner'] = df2['RoundWinner'].apply(lambda x: 1 if 'True' in x else 0)

In [ ]:
#Cambiamos datos true y false por nuemricos
df2['Survived'] = df2['Survived'].astype(int)


In [ ]:
#Cambiamos datos true y false por nuemricos
df2['AbnormalMatch'] = df2['AbnormalMatch'].astype(int)


In [ ]:
df2.head()

,Unnamed: 0,Map,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,TimeAlive,TravelledDistance,RLethalGrenadesThrown,RNonLethalGrenadesThrown,PrimaryAssaultRifle,PrimarySniperRifle,PrimaryHeavy,PrimarySMG,PrimaryPistol,FirstKillTime,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots,ArmaPrincipal,CounterTerrorist,Terrorist
0,600,1,1,4,1,0,1,0,0,51.120.248.995.704.500,10.083.140.737.457.000,0,4,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,750,4400,0,0,0,0,1,0,1
1,601,1,1,4,1,0,1,0,0,4.348.662.552.266.170,60.960.883.184.688.400,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,800,4400,0,0,0,0,1,0,1
2,602,1,1,4,1,0,1,0,0,3.735.469.847.054.540,5.642.252.383.582.750,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,1000,4400,0,0,0,0,1,0,1
3,603,1,1,4,1,0,1,0,0,4.767.886.136.441.470,8.963.473.313.229.970,0,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,850,4400,0,0,0,0,1,0,1
4,604,1,1,4,1,0,1,1,0,5.312.251.089.027.310,10.865.649.734.393.100,1,1,0.0,0.0,0.0,0.0,1,0.0,0,0,0,0,1000,4400,0,0,0,0,1,0,1


#UNA VEZ PREPARADOS LOS DATOS COMENZAMOS CON EL MODELO LINEAL SIMPLE#

In [ ]:
#Definir las variables independientes (X) y la variable dependiente (y)
X = df2[['MatchKills']]
y = df2[['MatchHeadshots']]

#Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Crear el modelo de regresión lineal
reg_model = LinearRegression()

#Entrenar el modelo
reg_model.fit(X_train, y_train)

#Realizar predicciones en el conjunto de prueba
y_pred = reg_model.predict(X_test)

#Limitar las predicciones entre 0 y 1
y_pred_clipped = np.clip(y_pred, 0, 1)

#Asignación de variable exclusiva para el input
y_pred_reglineal = y_pred_clipped

#Escalar los valores a un rango de 0 a 100
y_pred_scaled = y_pred_clipped

#Asegurar que las predicciones no superen el 100%

In [ ]:
# Variables predictoras e independiente
#Definir las variables independientes (X) y la variable dependiente (y)
X = df2[['MatchKills']]
y = df2[['MatchHeadshots']]

#Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Crear y entrenar el modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)



LinearRegression()

In [ ]:
# Predecir
y_pred = modelo.predict(X_test)

In [ ]:
# prompt: mostrar resultado con mean_squared_error mean_absolute_error r2_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2): {r2}')

Mean Squared Error (MSE): 3.280523206600969
Mean Absolute Error (MAE): 1.3032577080569405
R-squared (R2): 0.6916876368912634


 R² de 0.6917, lo que indica que aproximadamente el 69% de la variabilidad en los headshots por ronda puede explicarse a partir de las kills por partida. Este valor sugiere una relación lineal moderadamente fuerte entre ambas variables. En cuanto a los errores

  El Error Absoluto Medio (MAE) fue de 1.30, lo que implica que, en promedio, el modelo se equivoca por alrededor de 1.3 headshots en cada predicción. Por otro lado
  
  El Error Cuadrático Medio (MSE) fue de 3.28, lo que señala que existen errores más grandes ocasionales que afectan la precisión del modelo